# 609 Coursework 1

### Backtracking Algorithm

A backtracking algorithm attempts to solve a problem by recursively building candidate solutions. As it builds these solutions, if it becomes clear a partial candidate will no longer work, then it stops and retraces the solution path back until it can make a different choice that it has not tried yet.  

My backtracking function is built using the pseudocode provided. The inputs are the problem's specific parameters and then an initial partial candidate. It also requires 5 functions which are created for each specific problem. The functions are as follows: \
reject(P, c) - this will return TRUE if we need to stop using a partial candidate as it will not work, and returns FALSE otherwise \
accept(P, c) - this will reutrn TRUE if we have found a solution to the problem, and returns FALSE otherwise \
output(P, c) - this will return a solution to the problem once it has been found \
first(P, c) - given the current partial candidate, this function makes the first possible addition to this candidate \
next1(P, s) - given the current partial candidate that has just failed, it gives an alternative addition to this candidate given its previous attempt had to be stopped \

In [1]:
# INPUTS:
# P - this provides the particular parameters of the problem in question
# c - the initial partial candidate under consideration

backtracker <- function(P, c) {
  if (reject(P, c) == TRUE) { # move onto next start point
    return()
  }
  
  if (accept(P, c) == TRUE) { # add c to solution
    output(P, c) 
    return()
  }
  
  s <- first(P, c)
  
  while (!is.null(s)) {
    backtracker(P, s)
    s <- next1(P, s)
  }
}

### Integer Partition Problem

The aim of integer partitioning is to write a positive interger as the sum of other possible intergers, and we want to find all such partitions. We can do this via backtracking by recursively adding integers together until we exceed the target, then retrace our last step and add a smaller integer to the partition. We repeat this until we reach the target number exactly at which point we have a solution, but then we retrace the steps again and keep searching until we have search all possible solutions.

My backtracking approach is to start with an empty partition and keep adding 1 to it until you reach the target P (and then it prints the solution) or until you exceed P (then it stops). Once it exceeds P, it increases the final integer of the partition by 1 and checks its validity. It keeps increasing the final integer until the integer equals P, then it removes that integer. So our partial candidate is now just 1s, but with one less integer in it than before. Again it increases the final integer by one until it reaches P. And thus the recursive formula continues until the final partition will just be P itself. 

In [2]:
# INPUTS in backtracker function:
# P is the positive integer we want to partition
# c will be NULL as we start with an empty partition

# reject a candidate when the sum of the partition is greater than our target P
reject <- function(P, c) {
  if (sum(c) > P) {
    return(TRUE)
  } else {
    return(FALSE)
  }
}

# accept a candidate when the sum of the parrtition is equal to our target P
accept <- function(P, c) {
  if (sum(c) == P) {
    return(TRUE)
  } else {
    return(FALSE)
  }
}

# the first addition we make to a candidate is always to add '1' to the candidate partition
first <- function(P, c) {
  c <- c(c, 1)
  return(c)
}

# we want to print our solutions, but we want unique solutions not permutations of the same solution
# so we will only print solutions where the partition is written in ascending order
output <- function(P, c) {
  if (is.unsorted(c) == FALSE) { 
    print(c)
  }
}

# the next alternative addition to the partition is to add numbers other than '1' into the partition
# this function increases the last number in the candidate partition by 1, until that number is equal to our target P
next1 <- function(P, s) {
  # need to change s to the next alternative extension 
  last_digit <- s[length(s)]
  if (last_digit < P) {
    s[length(s)] <- last_digit + 1
  } else {
    s <- NULL  
  }
  return(s)
}

Here is an example where we have the partitions for P=10

In [3]:
backtracker(10,NULL)

 [1] 1 1 1 1 1 1 1 1 1 1
[1] 1 1 1 1 1 1 1 1 2
[1] 1 1 1 1 1 1 1 3
[1] 1 1 1 1 1 1 2 2
[1] 1 1 1 1 1 1 4
[1] 1 1 1 1 1 2 3
[1] 1 1 1 1 1 5
[1] 1 1 1 1 2 2 2
[1] 1 1 1 1 2 4
[1] 1 1 1 1 3 3
[1] 1 1 1 1 6
[1] 1 1 1 2 2 3
[1] 1 1 1 2 5
[1] 1 1 1 3 4
[1] 1 1 1 7
[1] 1 1 2 2 2 2
[1] 1 1 2 2 4
[1] 1 1 2 3 3
[1] 1 1 2 6
[1] 1 1 3 5
[1] 1 1 4 4
[1] 1 1 8
[1] 1 2 2 2 3
[1] 1 2 2 5
[1] 1 2 3 4
[1] 1 2 7
[1] 1 3 3 3
[1] 1 3 6
[1] 1 4 5
[1] 1 9
[1] 2 2 2 2 2
[1] 2 2 2 4
[1] 2 2 3 3
[1] 2 2 6
[1] 2 3 5
[1] 2 4 4
[1] 2 8
[1] 3 3 4
[1] 3 7
[1] 4 6
[1] 5 5
[1] 10


### Gray Code

To find a gray code, we can again use backtracking but it is of a different structure to that for integer partitioning. Here we are just searching for one possible solution, but it may take longer to travel down a path before we realise it is an incorrect one that we have to retrace. The idea here is to build up a gray code from a starting point by systematically changing one digit at a time until you either get a repeat (then you have to reject this candidate and retrace your path) or you reach a solution. 

For my backtracking algorithm, I start by changing the rightmost entry of the current candidate solution. If this is a repeated candidate, then I undo that and instead change the 2nd-right-most entry. I repeat this until I have a new candidate. With my new candidate I again start with the changing the right-most digit. This then recursively continues until either I have a full solution or until I have just repeats and no possible next candidates. Then I have to retrace further and change the digit one column to the left instead. 

In [4]:
# INPUTS in backtracker function:
# P is the length of our gray code 
# c is a 1xP matrix whose entries are 0s and 1s

# reject a candidate when it is a repeat of a previous line
reject <- function(P, c) {
  if (nrow(c) == 1) { # if the matrix c has just one row, then we don't reject it yet
    return(FALSE)
  } else {
    T_F <- logical(nrow(c) - 1)
    for (i in 1:(nrow(c) - 1)) {
      if (all(c[nrow(c), ] == c[i, ])) {   # if last row is identical with an earlier row, then TRUE
        T_F[i] <- TRUE
      } else {
        T_F[i] <- FALSE
      }
    }
    
    if (sum(T_F) > 0) { # if last row is identical with an earlier row, return TRUE to reject candidate
      return(TRUE)
    } else {
      return(FALSE)
    }
  }
}

# accept a candidate when the number of rows equals 2^P
accept <- function(P, c) {
  if (nrow(c) == 2^P) {
    return(TRUE)
  } else{
    return(FALSE)
  }
}

# we print our solution when it is found
# however, we only want one solution for this problem, so we then end the function via break 
# (however this does lead to an error as it is the incorrect usage of break(), so an improvement would be to fix this)
output <- function(P, c) {
  print(c)
  break()
}

# add a new row to matrix which is the same as previous last row, but with last entry switched
first <- function(P, c) {
  last_row <- c[nrow(c), ]
  last_row[P] <- 1 - last_row[P]
  c <- rbind(c, last_row)
  row.names(c) <- NULL
  return(c)
}

# given the current matrix in which the last row is a repeat of a previous row
# identify the column that is different between the last and second-to-last rows (column i)
# then delete this last row and instead change column i-1 of the second-to-last to be the new last row
next1 <- function(P, s) {
  last_row <- s[nrow(s), ]
  second_last_row <- s[nrow(s) - 1, ]
  diff_column <- which(last_row != second_last_row) 
  
  s <- s[-nrow(s),] #remove previous last row 
  new_last_row <- s[nrow(s), ]  
  new_last_row[diff_column-1] <- 1 - new_last_row[diff_column - 1] #create a new last row 
  s <- rbind(s, new_last_row)
  row.names(s) <- NULL
  
  return(s)
}

Here is an example where we have the gray code for P=4

In [5]:
backtracker(4,matrix(rep(0,4), nrow=1))

      [,1] [,2] [,3] [,4]
 [1,]    0    0    0    0
 [2,]    0    0    0    1
 [3,]    0    0    1    1
 [4,]    0    0    1    0
 [5,]    0    1    1    0
 [6,]    0    1    1    1
 [7,]    0    1    0    1
 [8,]    0    1    0    0
 [9,]    1    1    0    0
[10,]    1    1    0    1
[11,]    1    1    1    1
[12,]    1    1    1    0
[13,]    1    0    1    0
[14,]    1    0    1    1
[15,]    1    0    0    1
[16,]    1    0    0    0


ERROR: Error in output(P, c): no loop for break/next, jumping to top level


### Conclusions

The backtracker function is quite reusable as it provides the same framework for different functions and the structure of the inputs P, c can vary as is appropriate to the problem. Instead, you just have to create 5 specific functions for each different problem which is mangeable but could be quite time-consuming. 

There are definitely some improvements that could be made to the efficiency of both these algorithms. For the integer partition, there are more efficient cuts that could be made earlier for example so that it is not rechecking permutations of the same partition which could help reduce the time it takes to run. 

The gray code will only run for P less than 12 before you get a stack overflow error. This code also needs to be fixed so that it correctly exits the function after you have found one solution, whereas at the moment a break is used, but this is incorrect usage and so generates an error after printing the correct solution. However, the algorithm does successfully find a gray code for any appropriate starting vector which gives it more flexibility there. 